<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">COMP3611 - Building a Machine Learning Pipeline</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Marc de Kamps and University of Leeds</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.

# Building a Machine Learning Pipeline (Part 3)

## Objectives

Here, we finally analyse the data. We will experiment with different regressors, preform cross validation and parameter search and will use the *scikit-learn* interface to do so.

In particular, we will 
- Apply linear regression to make predictions
- Apply decision trees to make predictions
- Apply random forests to make predictions
- Compare these different regression methods ob the mean squared error criterion
- Perform cross validation and grid search

In [2]:
import os
import numpy as np
import pandas as pd
import tarfile

from sklearn.model_selection import StratifiedShuffleSplit

local_path = 'datasets/housing'

def restore():
    housing_tgz=tarfile.open(os.path.join(local_path,'./housing.tgz'))
    housing_tgz.extractall(path=local_path)
    housing_tgz.close()

    csv_path=os.path.join(local_path,'./housing.csv')
    housing = pd.read_csv(csv_path)

    # create test training set with stratified sampling (see previous notebook)
    housing["income_category"]=np.ceil(housing["median_income"]/1.5)
    housing["income_category"].where(housing["income_category"] < 5, 5.0, inplace = True)

    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=42)

    for train_index, test_index in split.split(housing,housing["income_category"]):
        strat_train_set = housing.loc[train_index]
        strat_test_set = housing.loc[test_index]
    
    for set_ in (strat_train_set, strat_test_set):
        set_.drop(("income_category"),axis=1,inplace=True)
        
   
    return strat_train_set, strat_test_set

strat_train_set, strat_test_set = restore()

/var/folders/tj/klxf1mr50kx14mdz29_1yw7w0000gr/T/ipykernel_31831/1634812113.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  housing_tgz.extractall(path=local_path)
/var/folders/tj/klxf1mr50kx14mdz29_1yw7w0000gr/T/ipykernel_31831/1634812113.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing["income_category"].where(housing["income_category"] < 5, 5.0, inplace = True)


In [3]:
housing=strat_train_set.drop("median_house_value",axis=1)
housing_labels=strat_train_set["median_house_value"].copy()

In [4]:
housing_num=housing.drop("ocean_proximity",axis=1)

In [5]:
num_attribs=list(housing_num)
cat_attribs=["ocean_proximity"]

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy="median")

In [8]:
from sklearn.base import BaseEstimator

class CombinedAttributesAdder(BaseEstimator):

    def __init__(self, do_add_bedrooms_per_room = False):
        
        # simply a binary variable per room
        self.do_add_bedrooms_per_room = do_add_bedrooms_per_room
        
        # These are the column indices of the respective columns. OK for illustration purposes.
        # For more robust code you would want to extract these values from the DataFrame by name.
        self.rooms_ix      = 3
        self.bedrooms_ix   = 4
        self.population_ix = 5
        self.household_ix  = 6
        
    def fit(self, X, y=None):
        # We don't transform the target values here
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:,self.rooms_ix]/X[:,self.household_ix]
        population_per_household = X[:, self.population_ix]/ X[:,self.rooms_ix]
        if self.do_add_bedrooms_per_room:
            bedrooms_per_room = X[:,self.bedrooms_ix]/X[:,self.rooms_ix]
            return np.c_[X,rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        

In [9]:
class DataFrameSelector(BaseEstimator):
    
    def __init__(self, attribute_names):
        self.attribute_names= attribute_names
        
    def fit(self,X, y = None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values



In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
num_pipeline= Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder',CombinedAttributesAdder()),
    ('std_scaler',StandardScaler())
])

In [12]:
cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attribs)),
    ('one hot',OneHotEncoder())
])

In [13]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline",num_pipeline),
    ("cat_pipeline",cat_pipeline)
])

In [14]:
housing_prepared = full_pipeline.fit_transform(housing)

In [15]:
housing_prepared

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 181632 stored elements and shape (16512, 15)>

In [16]:
housing_prepared.shape

(16512, 15)

### Regression
Up until this point, the pipeline is a succcint version of all we did in Part 2, with one small exception that we encourage you to find. *housing_prepared* is fully imputed and processed version of the test set. *housing_labels* the associated labesl.  The code for linear regression is shockingly simple.

In [17]:
from sklearn.linear_model import LinearRegression

lin_reg=LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


It is always a good idea to see what this looks like on the dataset:

In [18]:
some_data=housing.iloc[:5]
some_labels=housing_labels.iloc[:5]
some_data_prepared=full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

Predictions: [ 88370.9422656  304135.24477305 154102.43740032 183742.54725251
 247301.43479409]
Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


In [19]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse=mean_squared_error(housing_labels, housing_predictions)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

68916.20007477459


It does something sensible, but the root mean squared error is sizeable. Although some of this is caused by the deviation of prediction and outcome for the more expensive homes, inspection of individual predictions shows substantial deviations for some data points. And this is on the **training set**.

Let's try a decsion tree prediction

In [20]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()

tree_reg.fit(housing_prepared, housing_labels)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


Note the similarity of interface. Prediction also looks similar.

In [21]:
housing_predictions=tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rsme=np.sqrt(tree_mse)
tree_rsme

np.float64(0.0)

No error at all! Actually this is somewhat suspect. There is no doubt the tree has learnt the training set perfectly, but this may indicate overfitting.

### Cross validation
In cross validation the dataset is partitioned. In n-fold cross validation, the following experimnt is repeated n times: set a fraction of n-1/n patterns apart from training and use the rest for evaluation. This gives n different scores. THe implementation is straightforward with only one caveat: *scikit-learn* expects a utility function (lower is better than higher) rather than a cost function, and the scoring function is the negative of the MSE. The code below compensates for that.

In [22]:
from sklearn.model_selection import cross_val_score

scores =cross_val_score(tree_reg, housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)

tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print('Scores:',scores)
    print('Mean:',scores.mean())
    print('Standard deviation:',scores.std())
    
display_scores(tree_rmse_scores)

Scores: [68471.8964312  66798.76354078 67395.35238329 71256.51948013
 66513.4900307  77338.97898887 72331.41035852 71835.18724402
 68526.68897173 72068.04826284]
Mean: 70253.63356920729
Standard deviation: 3187.004326490138


If anything, the performance is now slightly worse than for linear regression. This is a clear example of overfitting by the original tree.

**Exercise** Carry out cross validation for the linear regression

Now, we will try a random forest regressors.

In [23]:
from sklearn.ensemble import RandomForestRegressor
forest_reg=RandomForestRegressor()
forest_reg.fit(housing_prepared,housing_labels)

housing_predictions=forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rsme=np.sqrt(forest_mse)
forest_rsme

np.float64(18293.51042863996)

This looks encouraging: a better error than linear regression, but no clear overfitting. Again, let's cross validate.

In [24]:
scores=cross_val_score(forest_reg, housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores=np.sqrt(-scores)

display_scores(forest_rmse_scores)

Scores: [50237.77936446 47655.40009607 46399.31910998 50711.45597832
 47317.27144909 49627.62375545 50324.83707848 48791.13916278
 46802.07044627 52785.62563342]
Mean: 49065.2522074313
Standard deviation: 1928.1143549146097


## Grid Search

This is clearly better than the linear regressor. We have used the random forest 'out of the box', not bothering to tweak its parameters. The lectures should have given you some ideas on how different parameter settings can be used to alter decision trees and random forests. In a *grid search* you systematically try out combinations of relevant parameters. As you can check from the *scikit-learn* documentation, you can vary (at least) the following parameters:

- n_estimators
- max_features
- bootstrap

Suppose you want to systematically explore a number of parameter settings, then *GridSearchCV* can help you do this:

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid  = [{'n_estimators': [3, 10, 30],'max_features':[2, 4, 6, 8]}, 
               {'bootstrap': [False],'n_estimators':[3,10],'max_features':[2,3,4]}]

forest_reg=RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

,estimator,RandomForestRegressor()
,param_grid,"[{'max_features': [2, 4, ...], 'n_estimators': [3, 10, ...]}, {'bootstrap': [False], 'max_features': [2, 3, ...], 'n_estimators': [3, 10]}]"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,30


After this has finished, you need to get the best scoring model:

In [27]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [28]:
grid_search.best_estimator_

,n_estimators,30
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,8
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


The evaluation scores are available:

In [29]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

63611.03434134411 {'max_features': 2, 'n_estimators': 3}
54714.5471137217 {'max_features': 2, 'n_estimators': 10}
52464.389732684256 {'max_features': 2, 'n_estimators': 30}
59674.772306596504 {'max_features': 4, 'n_estimators': 3}
52302.72712070335 {'max_features': 4, 'n_estimators': 10}
50149.14949138804 {'max_features': 4, 'n_estimators': 30}
57846.99530917314 {'max_features': 6, 'n_estimators': 3}
51421.01718286841 {'max_features': 6, 'n_estimators': 10}
49614.09666394478 {'max_features': 6, 'n_estimators': 30}
58343.11043085119 {'max_features': 8, 'n_estimators': 3}
51080.66042997501 {'max_features': 8, 'n_estimators': 10}
49399.75330781697 {'max_features': 8, 'n_estimators': 30}
61717.524620812685 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54040.66649136939 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59014.73078531052 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52157.04320202833 {'bootstrap': False, 'max_features': 3, 'n_estimators': 

You should explore the possibility to do a **RandomizedSearchCV**.

**Exercise** Why is this sometimes a better option? When would you use it?

### Feature Importance

It can be useful to explore feature importance:

In [30]:
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs= ["rooms_per_hhold", "pop_per_hhold","bedrooms_per_room"]

# Here we use the OneHotEncoder class to retrieve the original categories
encoder=OneHotEncoder()
encoder.fit(housing[cat_attribs])
cat_one_hot_attribs = [ el for el in encoder.categories_[0]]
print(cat_one_hot_attribs)
attributes=num_attribs + extra_attribs + cat_one_hot_attribs


['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']


In [31]:
sorted(zip(feature_importances,attributes),reverse=True)

[(np.float64(0.32818383644745514), 'median_income'),
 (np.float64(0.1655131204534893), '<1H OCEAN'),
 (np.float64(0.16429667951659702), 'pop_per_hhold'),
 (np.float64(0.08490281737721701), 'longitude'),
 (np.float64(0.08179455695202853), 'latitude'),
 (np.float64(0.05477856848435853), 'rooms_per_hhold'),
 (np.float64(0.03976775612029077), 'housing_median_age'),
 (np.float64(0.016500485553626834), 'total_bedrooms'),
 (np.float64(0.016404915864260065), 'population'),
 (np.float64(0.016163335710851653), 'total_rooms'),
 (np.float64(0.015107135077422177), 'households'),
 (np.float64(0.01171475681313572), 'bedrooms_per_room'),
 (np.float64(0.0028550511550194525), 'NEAR BAY'),
 (np.float64(0.0019835912512381143), 'ISLAND'),
 (np.float64(3.339322300952557e-05), 'INLAND')]

Interestingly, the feature that a house must be close to the ocean seems to be the only important categorical feature. You can consider leaving out the the others.


## Evaluation on the test set

This is a step you always should do. If the results are substantially worse than the cross validated ones, you still must distrust your model.

In [32]:
final_model=grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value",axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions=final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test,final_predictions)
final_rsme = np.sqrt(final_mse)
print(final_rsme)

46659.79281349502


This looks reasonably close to the cross validated results on the training set.